In [ ]:
!pip install -q transformers huggingface_hub


In [1]:
%%bash
mkdir -p /kaggle/working/demo_vulns

cat > /kaggle/working/demo_vulns/app_vuln.py <<'PY'
import sqlite3
import subprocess
import pickle
import hashlib
import os

API_KEY = "ABCD1234EFGH5678IJKL9012MNOPQRST"

def setup_db(db=":memory:"):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute("CREATE TABLE users(id INTEGER PRIMARY KEY, username TEXT, password TEXT)")
    c.execute("INSERT INTO users(username,password) VALUES ('alice','alicepass')")
    c.execute("INSERT INTO users(username,password) VALUES ('bob','bobpass')")
    conn.commit()
    return conn

def find_user_bad(conn, username):
    q = "SELECT id, username FROM users WHERE username = '" + username + "'"
    print("Running SQL:", q)
    return conn.execute(q).fetchall()

def run_shell_bad(cmd):
    subprocess.run("ls " + cmd, shell=True)

def unsafe_deserialize(data_bytes):
    obj = pickle.loads(data_bytes)
    return obj

def weak_hash(password):
    return hashlib.md5(password.encode()).hexdigest()

def generate_token_bad():
    return "token-" + str(os.getpid())

def log_secret():
    print("Using API key:", API_KEY)
PY

cat > /kaggle/working/demo_vulns/auth_vuln.py <<'PY'
import sqlite3

JWT_SECRET = "secret_jwt_key_1234567890"

def check_login(conn, username, password):
    row = conn.execute("SELECT password FROM users WHERE username = ?", (username,)).fetchone()
    if not row:
        return False
    stored = row[0]
    return stored == password

def get_user_by_search(conn, q):
    sql = "SELECT id, username FROM users WHERE username LIKE '%" + q + "%'"
    return conn.execute(sql).fetchall()
PY

cat > /kaggle/working/demo_vulns/web_vuln.js <<'JS'
// demo_vulns/web_vuln.js
function showMessage(userHtml) {
  document.getElementById("out").innerHTML = "<p>" + userHtml + "</p>";
}

function runCode(codeStr) {
  eval(codeStr);
}

const CLIENT_API_KEY = "sk_test_1234567890SECRET";

function debugLog(user) {
  console.log("Logging user and key:", user, CLIENT_API_KEY);
}
JS

cat > /kaggle/working/demo_vulns/config.json <<'JSON'
{
  "db_password": "supersecretpassword123",
  "aws_access_key": "AKIAEXAMPLEKEY12345678",
  "s3_bucket": "my-app-bucket"
}
JSON

cat > /kaggle/working/demo_vulns/misc_vuln.py <<'PY'
import os
from http.server import BaseHTTPRequestHandler, HTTPServer

class SimpleHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"ok")

def start_server():
    server = HTTPServer(('0.0.0.0', 8080), SimpleHandler)
    print("Serving on :8080")
    server.serve_forever()

def insecure_permissions(path):
    with open(path, "w") as f:
        f.write("secret stuff")
    os.chmod(path, 0o666)
PY

echo "demo_vulns created at /kaggle/working/demo_vulns"
ls -la /kaggle/working/demo_vulns


demo_vulns created at /kaggle/working/demo_vulns
total 28
drwxr-xr-x 2 root root 4096 Sep 17 19:59 .
drwxr-xr-x 4 root root 4096 Sep 17 19:59 ..
-rw-r--r-- 1 root root 1025 Sep 17 19:59 app_vuln.py
-rw-r--r-- 1 root root  435 Sep 17 19:59 auth_vuln.py
-rw-r--r-- 1 root root  124 Sep 17 19:59 config.json
-rw-r--r-- 1 root root  486 Sep 17 19:59 misc_vuln.py
-rw-r--r-- 1 root root  324 Sep 17 19:59 web_vuln.js


In [2]:
# Scanner definitions (regex + Python AST)
import os, re, ast, json, html
from math import log2
from collections import namedtuple
from IPython.display import display, HTML as IPHTML

Finding = namedtuple("Finding", ["id","file","start_line","end_line","lang","detector","snippet","raw"])

REGEX_DETECTORS = [
    ("HardcodedSecret", re.compile(r"(?:api[_-]?key|secret|password|AKIA|sk_test)[\"']?\s*[:=]?\s*[\"']?[A-Za-z0-9-_@./+]{16,}?[\"']?", re.I)),
    ("JS_eval", re.compile(r"\beval\s*\(", re.I)),
    ("JS_innerHTML", re.compile(r"\.(innerHTML|outerHTML|insertAdjacentHTML)\b", re.I)),
    ("SQL_concat", re.compile(r"(SELECT|UPDATE|INSERT|DELETE).*\+.*[a-zA-Z0-9_$]", re.I|re.S)),
    ("pickle_loads", re.compile(r"\bpickle\.(loads|load)\b", re.I)),
    ("md5_hash", re.compile(r"\bhashlib\.md5\b", re.I)),
    ("subprocess_shell", re.compile(r"subprocess\.(run|Popen|call)\([^)]*shell\s*=\s*True", re.I)),
    ("insecure_file_perm", re.compile(r"os\.chmod\([^,]*,\s*0o?6{2,3}\b", re.I)),
]

HIGH_ENTROPY_LITERAL_RE = re.compile(r"['\"]([A-Za-z0-9+/=]{30,})['\"]")

def file_entropy(s: str) -> float:
    if not s:
        return 0.0
    freq = {}
    for ch in s:
        freq[ch] = freq.get(ch,0)+1
    ent = 0.0
    L = len(s)
    for v in freq.values():
        p = v / L
        if p>0:
            ent -= p * (log2(p))
    return ent

def make_id(file, line, detector):
    return f"{os.path.basename(file)}:{line}:{detector}"

class PyAstDetector(ast.NodeVisitor):
    def __init__(self, text):
        self.findings = []
        self.text_lines = text.splitlines()
    def snippet_for(self, node, context=2):
        try:
            start = node.lineno - 1
            end = getattr(node, "end_lineno", node.lineno) - 1
            first = max(0, start - context)
            last = min(len(self.text_lines)-1, end + context)
            return "\n".join(self.text_lines[first:last+1])
        except Exception:
            return ""
    def visit_Call(self, node):
        func = node.func
        func_name = ""
        if isinstance(func, ast.Attribute):
            func_name = func.attr
        elif isinstance(func, ast.Name):
            func_name = func.id
        for arg in node.args:
            if isinstance(arg, ast.BinOp) and isinstance(arg.op, ast.Add):
                snip = self.snippet_for(node)
                lineno = getattr(node, "lineno", 0)
                self.findings.append(dict(detector="Py_AST_concat_call", lineno=lineno, snippet=snip, func=func_name))
        if func_name in ("eval","exec"):
            snip = self.snippet_for(node)
            lineno = getattr(node, "lineno", 0)
            self.findings.append(dict(detector="Py_AST_eval_exec", lineno=lineno, snippet=snip, func=func_name))
        self.generic_visit(node)

def scan_file(path):
    findings=[]
    try:
        text = open(path,"r",encoding="utf-8",errors="ignore").read()
    except Exception:
        return findings
    _, ext = os.path.splitext(path.lower())
    lang = "python" if ext==".py" else ("javascript" if ext in (".js",".jsx",".ts",".tsx") else "other")
    # regex detectors
    for name, regex in REGEX_DETECTORS:
        for m in regex.finditer(text):
            start_line = text.count("\n",0,m.start()) + 1
            snippet = get_snippet_with_context(text, m.start(), m.end(), context_chars=200)
            findings.append(Finding(make_id(path,start_line,name), path, start_line, start_line, lang, name, snippet, m.group(0)))
    # high entropy literals
    for m in HIGH_ENTROPY_LITERAL_RE.finditer(text):
        val = m.group(1)
        if file_entropy(val) >= 4.0:
            start_line = text.count("\n",0,m.start()) + 1
            findings.append(Finding(make_id(path,start_line,"HighEntropyLiteral"), path, start_line, start_line, "other", "HighEntropyLiteral", val[:200], val))
    # python AST
    if lang=="python":
        try:
            tree = ast.parse(text)
            visitor = PyAstDetector(text)
            visitor.visit(tree)
            for d in visitor.findings:
                lineno = d["lineno"]
                findings.append(Finding(make_id(path,lineno,d["detector"]), path, lineno, lineno, lang, d["detector"], d["snippet"], ""))
        except Exception:
            pass
    return findings

# helper to grab a snippet with exact source and preserve original line numbers for display
def get_snippet_with_context(text, start_idx, end_idx, context_chars=200):
    start = max(0, start_idx - context_chars)
    end = min(len(text), end_idx + context_chars)
    return text[start:end]

def scan_directory(root):
    all_findings=[]
    for dirpath, dirs, files in os.walk(root):
        if any(skip in dirpath for skip in ("/.git","/node_modules","/__pycache__")):
            continue
        for fname in files:
            if not fname.lower().endswith((".py",".js",".json",".yaml",".yml")):
                continue
            full = os.path.join(dirpath, fname)
            all_findings.extend(scan_file(full))
    return all_findings

# quick run (you can re-run after creating files)
project_path = "/kaggle/working/demo_vulns"
findings = scan_directory(project_path)
print(f"Scanner found {len(findings)} findings (raw).")
# show brief
for f in findings:
    print(f"{f.file}:{f.start_line} [{f.detector}]")


Scanner found 15 findings (raw).
/kaggle/working/demo_vulns/auth_vuln.py:3 [HardcodedSecret]
/kaggle/working/demo_vulns/auth_vuln.py:6 [SQL_concat]
/kaggle/working/demo_vulns/web_vuln.js:10 [HardcodedSecret]
/kaggle/working/demo_vulns/web_vuln.js:7 [JS_eval]
/kaggle/working/demo_vulns/web_vuln.js:3 [JS_innerHTML]
/kaggle/working/demo_vulns/misc_vuln.py:18 [insecure_file_perm]
/kaggle/working/demo_vulns/app_vuln.py:7 [HardcodedSecret]
/kaggle/working/demo_vulns/app_vuln.py:13 [SQL_concat]
/kaggle/working/demo_vulns/app_vuln.py:27 [pickle_loads]
/kaggle/working/demo_vulns/app_vuln.py:31 [md5_hash]
/kaggle/working/demo_vulns/app_vuln.py:24 [subprocess_shell]
/kaggle/working/demo_vulns/app_vuln.py:7 [HighEntropyLiteral]
/kaggle/working/demo_vulns/app_vuln.py:24 [Py_AST_concat_call]
/kaggle/working/demo_vulns/config.json:2 [HardcodedSecret]
/kaggle/working/demo_vulns/config.json:3 [HardcodedSecret]


In [3]:
# AI triage and suggestion enrichment
from transformers import pipeline
import time

print("Loading AI classifier (may take a bit if not cached)...")
# If you have model-download problems, replace the model string with a smaller one or local path
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
print("Model loaded.")

CATEGORIES = [
    "High risk: Remote Code Execution",
    "High risk: SQL Injection",
    "High risk: Hardcoded Secrets",
    "Medium risk: Weak cryptography",
    "Medium risk: Cross Site Scripting (XSS)",
    "Low risk: Insecure config or logging"
]

# Human friendly explanation & remediation templates mapped to categories
SUGGESTION_TEMPLATES = {
    "High risk: Remote Code Execution": {
        "explanation": "This code appears to execute or forward user-controlled data to a shell or use unsafe deserialization/eval.",
        "fix": "Avoid shell=True and passing concatenated strings to subprocess. Use argument lists (subprocess.run([...])) or validate/whitelist input. Replace pickle.loads with safe parsers or signed payloads."
    },
    "High risk: SQL Injection": {
        "explanation": "This snippet builds SQL queries via string concatenation or unparameterized patterns.",
        "fix": "Use parameterized queries or prepared statements (e.g. cursor.execute(query, (param,))). Consider using an ORM to avoid manual SQL building."
    },
    "High risk: Hardcoded Secrets": {
        "explanation": "A long/high-entropy literal or an obvious API key/secret is present in code/config.",
        "fix": "Remove secrets from source code. Use environment variables, a secrets manager, or platform secret storage. Rotate any keys that were committed."
    },
    "Medium risk: Weak cryptography": {
        "explanation": "Weak hashing or insecure randomness detected (e.g. MD5, predictable tokens).",
        "fix": "Use bcrypt or Argon2 for password hashing. Use secrets.token_urlsafe() for tokens and modern cryptographic libraries for integrity."
    },
    "Medium risk: Cross Site Scripting (XSS)": {
        "explanation": "Unescaped HTML insertion (innerHTML) or eval in client context can lead to XSS.",
        "fix": "Sanitize or escape user input, use textContent instead of innerHTML, and avoid eval(). Use a CSP header."
    },
    "Low risk: Insecure config or logging": {
        "explanation": "This looks like insecure permissions or logging of potentially sensitive data.",
        "fix": "Tighten file permissions, redact secrets from logs, and avoid storing credentials in cleartext."
    }
}

def ai_enrich(findings):
    enriched=[]
    for f in findings:
        prompt_text = f"{f.detector}: {f.snippet[:400]}"
        try:
            out = classifier(prompt_text, CATEGORIES)
            top_label = out["labels"][0]
            top_score = float(out["scores"][0])
        except Exception:
            top_label = None
            top_score = 0.0

        if top_label and "High risk" in top_label:
            ai_sev = "High"
        elif top_label and "Medium risk" in top_label:
            ai_sev = "Medium"
        else:
            ai_sev = "Low"

        templates = SUGGESTION_TEMPLATES.get(top_label, {
            "explanation": "Manual review required. The detector flagged this snippet.",
            "fix": "Inspect the snippet and apply appropriate secure coding practice."
        })

        enriched.append({
            "id": f.id,
            "file": f.file,
            "start_line": f.start_line,
            "end_line": f.end_line,
            "lang": f.lang,
            "detector": f.detector,
            "snippet": f.snippet,
            "raw": f.raw,
            "ai_category": top_label,
            "ai_confidence": round(top_score,3),
            "ai_severity": ai_sev,
            "ai_explanation": templates["explanation"],
            "ai_fix_suggestion": templates["fix"]
        })
    return enriched

print("Running AI enrichment ...")
t0 = time.time()
ai_findings = ai_enrich(findings)
print("AI enrichment done in", int(time.time()-t0), "s. Findings:", len(ai_findings))


2025-09-17 20:00:28.834098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758139229.139912      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758139229.229673      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading AI classifier (may take a bit if not cached)...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Model loaded.
Running AI enrichment ...
AI enrichment done in 110 s. Findings: 15


In [4]:
# Save AI-enriched JSON
import json, html
out_json = "/kaggle/working/results_ai.json"
with open(out_json, "w", encoding="utf-8") as fh:
    json.dump(ai_findings, fh, indent=2)
print("Saved AI JSON ->", out_json)

# Build HTML report with numbered snippet lines
def numbered_snippet_html(file_path, snippet, highlight_line=None):
    # To get correct line numbers we try to map snippet to the original file lines
    try:
        with open(file_path, "r", encoding="utf-8", errors="ignore") as fh:
            full = fh.read()
    except Exception:
        full = ""
    # find snippet location in file if possible
    idx = full.find(snippet.strip())
    # fallback: use snippet as-is
    if idx != -1:
        # compute start line
        start_line = full.count("\n", 0, idx) + 1
        # extract exact lines from file around snippet length
        lines = full.splitlines()
        # choose lines covering the snippet roughly: from start_line-3 to start_line+len(snippet_lines)+3
        snippet_lines = snippet.splitlines()
        s = max(0, start_line-3)
        e = min(len(lines), start_line - 1 + len(snippet_lines) + 3)
        chosen = lines[s:e]
        first_line_no = s + 1
    else:
        chosen = snippet.splitlines()
        first_line_no = 1

    # produce HTML with numbered lines
    html_lines = []
    for i, ln in enumerate(chosen, start=first_line_no):
        safe = html.escape(ln)
        if highlight_line and i == highlight_line:
            html_lines.append(f'<div style="background:#ffe8e8"><code>{i:4d}: {safe}</code></div>')
        else:
            html_lines.append(f'<div><code>{i:4d}: {safe}</code></div>')
    return "\n".join(html_lines)

# Build rows
rows = []
for r in ai_findings:
    snippet_html = numbered_snippet_html(r["file"], r["snippet"], highlight_line=r["start_line"])
    rows.append(f"""
    <tr>
      <td style="vertical-align:top">{html.escape(r['file'])}:{r['start_line']}</td>
      <td style="vertical-align:top">{html.escape(r['detector'])}</td>
      <td style="vertical-align:top">{r['ai_severity']}</td>
      <td style="vertical-align:top">{r['ai_confidence']}</td>
      <td style="vertical-align:top">{html.escape(str(r.get('ai_category','')))}</td>
      <td style="vertical-align:top"><b>Explanation:</b> {html.escape(r['ai_explanation'])}<br/><b>Fix:</b> {html.escape(r['ai_fix_suggestion'])}</td>
      <td style="vertical-align:top; max-width:600px">{snippet_html}</td>
    </tr>
    """)

html_doc = f"""
<html><head><meta charset="utf-8"><title>AI-enriched Scan Results</title>
<style>
  table {{ border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; }}
  th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
  th {{ background: #f2f2f2; }}
  code {{ white-space: pre; font-family: monospace; }}
</style>
</head><body>
<h1>AI-enriched Scan Results</h1>
<p>File: /kaggle/working/demo_vulns — Total findings: {len(ai_findings)}</p>
<table>
<tr><th>Location</th><th>Detector</th><th>AI Sev</th><th>AI Conf</th><th>AI Category</th><th>AI Explanation & Fix</th><th>Snippet (with line numbers)</th></tr>
{''.join(rows)}
</table>
</body></html>
"""

out_html = "/kaggle/working/results_ai.html"
with open(out_html, "w", encoding="utf-8") as fh:
    fh.write(html_doc)
print("Saved HTML ->", out_html)

# Display HTML inline in notebook
display(IPHTML(html_doc))


Saved AI JSON -> /kaggle/working/results_ai.json
Saved HTML -> /kaggle/working/results_ai.html


Location,Detector,AI Sev,AI Conf,AI Category,AI Explanation & Fix,Snippet (with line numbers)
/kaggle/working/demo_vulns/auth_vuln.py:3,HardcodedSecret,High,0.376,High risk: Hardcoded Secrets,"Explanation: A long/high-entropy literal or an obvious API key/secret is present in code/config.Fix: Remove secrets from source code. Use environment variables, a secrets manager, or platform secret storage. Rotate any keys that were committed.","1: import sqlite3 2: 3: JWT_SECRET = ""secret_jwt_key_1234567890"" 4: 5: def check_login(conn, username, password): 6: row = conn.execute(""SELECT password FROM users WHERE username = ?"", (username,)).fetchone() 7: if not row: 8: return False 9: stored = row[0] 10: return stored == password 11: 12: def get_user_by_search(conn, q):"
/kaggle/working/demo_vulns/auth_vuln.py:6,SQL_concat,Medium,0.367,Medium risk: Weak cryptography,"Explanation: Weak hashing or insecure randomness detected (e.g. MD5, predictable tokens).Fix: Use bcrypt or Argon2 for password hashing. Use secrets.token_urlsafe() for tokens and modern cryptographic libraries for integrity.","1: import sqlite3 2: 3: JWT_SECRET = ""secret_jwt_key_1234567890"" 4: 5: def check_login(conn, username, password): 6: row = conn.execute(""SELECT password FROM users WHERE username = ?"", (username,)).fetchone() 7: if not row: 8: return False 9: stored = row[0] 10: return stored == password 11: 12: def get_user_by_search(conn, q): 13: sql = ""SELECT id, username FROM users WHERE username LIKE '%"" + q + ""%'"" 14: return conn.execute(sql).fetchall()"
/kaggle/working/demo_vulns/web_vuln.js:10,HardcodedSecret,High,0.356,High risk: Hardcoded Secrets,"Explanation: A long/high-entropy literal or an obvious API key/secret is present in code/config.Fix: Remove secrets from source code. Use environment variables, a secrets manager, or platform secret storage. Rotate any keys that were committed.","1: // demo_vulns/web_vuln.js 2: function showMessage(userHtml) { 3: document.getElementById(""out"").innerHTML = ""<p>"" + userHtml + ""</p>""; 4: } 5: 6: function runCode(codeStr) { 7: eval(codeStr); 8: } 9: 10: const CLIENT_API_KEY = ""sk_test_1234567890SECRET""; 11: 12: function debugLog(user) { 13: console.log(""Logging user and key:"", user, CLIENT_API_KEY); 14: }"
/kaggle/working/demo_vulns/web_vuln.js:7,JS_eval,Low,0.293,Low risk: Insecure config or logging,"Explanation: This looks like insecure permissions or logging of potentially sensitive data.Fix: Tighten file permissions, redact secrets from logs, and avoid storing credentials in cleartext.","1: // demo_vulns/web_vuln.js 2: function showMessage(userHtml) { 3: document.getElementById(""out"").innerHTML = ""<p>"" + userHtml + ""</p>""; 4: } 5: 6: function runCode(codeStr) { 7: eval(codeStr); 8: } 9: 10: const CLIENT_API_KEY = ""sk_test_1234567890SECRET""; 11: 12: function debugLog(user) { 13: console.log(""Logging user and key:"", user, CLIENT_API_KEY); 14: }"
/kaggle/working/demo_vulns/web_vuln.js:3,JS_innerHTML,Medium,0.266,Medium risk: Weak cryptography,"Explanation: Weak hashing or insecure randomness detected (e.g. MD5, predictable tokens).Fix: Use bcrypt or Argon2 for password hashing. Use secrets.token_urlsafe() for tokens and modern cryptographic libraries for integrity.","1: // demo_vulns/web_vuln.js 2: function showMessage(userHtml) { 3: document.getElementById(""out"").innerHTML = ""<p>"" + userHtml + ""</p>""; 4: } 5: 6: function runCode(codeStr) { 7: eval(codeStr); 8: } 9: 10: const CLIENT_API_KEY = ""sk_test_1234567890SECRET""; 11: 12: function debugLog(user) { 13: console.log(""Logging user and key:"", user, CLIENT_API_KEY); 14: }"
/kaggle/working/demo_vulns/misc_vuln.py:18,insecure_file_perm,Medium,0.284,Medium risk: Weak cryptography,"Explanation: Weak hashing or insecure randomness detected (e.g. MD5, predictable tokens).Fix: Use bcrypt or Argon2 for password hashing. Use secrets.token_urlsafe() for tokens and modern cryptographic libraries for integrity.","9: 10: def start_server()

In [5]:
import pandas as pd
df = pd.DataFrame(ai_findings)
# flatten snippet newline for CSV
df['snippet_short'] = df['snippet'].apply(lambda s: s.replace("\n","\\n")[:1000])
csv_path = "/kaggle/working/results_ai.csv"
df.to_csv(csv_path, index=False)
print("Saved CSV ->", csv_path)

# Save Excel as well
xlsx_path = "/kaggle/working/results_ai.xlsx"
df.to_excel(xlsx_path, index=False)
print("Saved XLSX ->", xlsx_path)

# show top rows
display(df.head())


Saved CSV -> /kaggle/working/results_ai.csv
Saved XLSX -> /kaggle/working/results_ai.xlsx


,id,file,start_line,end_line,lang,detector,snippet,raw,ai_category,ai_confidence,ai_severity,ai_explanation,ai_fix_suggestion,snippet_short
0,auth_vuln.py:3:HardcodedSecret,/kaggle/working/demo_vulns/auth_vuln.py,3,3,python,HardcodedSecret,"import sqlite3\n\nJWT_SECRET = ""secret_jwt_key...","SECRET = ""secret_jwt_key_1",High risk: Hardcoded Secrets,0.376,High,A long/high-entropy literal or an obvious API ...,Remove secrets from source code. Use environme...,"import sqlite3\n\nJWT_SECRET = ""secret_jwt_key..."
1,auth_vuln.py:6:SQL_concat,/kaggle/working/demo_vulns/auth_vuln.py,6,6,python,SQL_concat,"import sqlite3\n\nJWT_SECRET = ""secret_jwt_key...","SELECT password FROM users WHERE username = ?""...",Medium risk: Weak cryptography,0.367,Medium,Weak hashing or insecure randomness detected (...,Use bcrypt or Argon2 for password hashing. Use...,"import sqlite3\n\nJWT_SECRET = ""secret_jwt_key..."
2,web_vuln.js:10:HardcodedSecret,/kaggle/working/demo_vulns/web_vuln.js,10,10,javascript,HardcodedSecret,// demo_vulns/web_vuln.js\nfunction showMessag...,"API_KEY = ""sk_test_12345678",High risk: Hardcoded Secrets,0.356,High,A long/high-entropy literal or an obvious API ...,Remove secrets from source code. Use environme...,// demo_vulns/web_vuln.js\nfunction showMessag...
3,web_vuln.js:7:JS_eval,/kaggle/working/demo_vulns/web_vuln.js,7,7,javascript,JS_eval,// demo_vulns/web_vuln.js\nfunction showMessag...,eval(,Low risk: Insecure config or logging,0.293,Low,This looks like insecure permissions or loggin...,"Tighten file permissions, redact secrets from ...",// demo_vulns/web_vuln.js\nfunction showMessag...
4,web_vuln.js:3:JS_innerHTML,/kaggle/working/demo_vulns/web_vuln.js,3,3,javascript,JS_innerHTML,// demo_vulns/web_vuln.js\nfunction showMessag...,.innerHTML,Medium risk: Weak cryptography,0.266,Medium,Weak hashing or insecure randomness detected (...,Use bcrypt or Argon2 for password hashing. Use...,// demo_vulns/web_vuln.js\nfunction showMessag...
